# Evolutionary Dynamics in the Lotka-Volterra Model

In this example, we explore a very simple model for evolutionary dynamics within the context of a spatially extended stochastic Lotka-Volterra system. In particular, we shall change the predator-prey interaction from a fixed rate to being dependent on traits of both the participating individual species members.

To start, we need to load the `pymes` library. We only load what we need so not to clutter our environment.

In [ ]:
from pymes import World, Species, BirthReaction, DeathReaction, PredationBirthReaction, Hop, Occupant
from pymes.pymes import Site

import numpy as np

In contrast to the previous examples, here we will need to modify our species, occupant and reaction objects to allow for having traits, passing them on to offspring with some variation and allowing the interaction reaction to be dependent on these traits.

First of all, we need a way for new trait values to be chosen. Our trait values will be from the interval $(0, 1)$. Here, we use a normal distribution that is truncated to this interval, a mean that is set to the parent's trait value and a width that controls how much an offspring's trait value can deviate from its parent's trait value. This is a very simply choice, that's easy to understand and already leads to interesting results. However, other options are possible, for example using a symmetrized Beta distribution.

In [ ]:
def truncated_normal(rng: np.random.Generator, mean: float, width: float):
    while True:
        r = np.random.normal(mean, width)
        if r > 0 and r < 1:
            return r

Next, we subclass the `Occupant` class to allow for `Occupant`s to have `trait` values. The initializer of this new class has a new `trait` argument that gets set on the new object after calling the `Occupant` class' initializer. 

In [ ]:
class TraitOccupant(Occupant):
    def __init__(self, species: Species, initial_site: Site, trait: float):
        Occupant.__init__(self, species, initial_site)
        self.trait = trait

We also need to subclass `Species` to implement the whole `trait` functionality. We introduce a new `EvolvingSpecies` subclass that has two new parameters, `mutation_parameter` and `initial_trait`. The `mutation_parameter` sets the width of the truncated normal distribution and thus controls the "mutation rate". The `inital_trait` parameter simply sets the initial trait value at the start of a simulation run (0.5 is a good choice).

We now have to also slightly change how new occupants are created. Therefore, we need to override the `create_occupant` member function of the original `Species` class. This new function gets passed the parent occupant's trait value and receives a new trait value from the truncated normal distribution. If the parent trait value is not set (i.e. it is `None`), the `initial_trait` value is used. The function then returns a new `TraitOccupant` object with the new trait value at the given site.

Finally, we introduce a member function called `trait_frequencies`. This allows us to extract a snapshot of the distribution of trait values, which illustrates how the two species adapt their traits over time.

In [ ]:
class EvolvingSpecies(Species):
    def __init__(self, name: str, mutation_parameter: float, initial_trait: float):
        Species.__init__(self, name)
        self.mutation_parameter = mutation_parameter
        self.initial_trait = initial_trait

    def create_occupant(self, initial_site: Site, parent: TraitOccupant | None=None, rng: np.random.Generator=None) -> Occupant:
        if parent is None:
            trait = self.initial_trait
        else:
            trait = truncated_normal(rng, parent.trait, self.mutation_parameter)
        return TraitOccupant(self, initial_site, trait)
    
    def trait_frequencies(self, nr_bins: int=51):
        members: list[TraitOccupant] = self.members
        H, _ = np.histogram(
            [occupant.trait for occupant in members],
            bins=nr_bins,
            range=(0, 1),
            density=True,
        )
        return H


The last thing we need to change is how the predation birth reaction decides if the reaction takes place or not. The `PredationBirthReaction` class simply generates a random number and compares this with the given rate. We now subclass to create `TraitPredationBirthReaction` and override the `decide` member function. This new version calculates a combined rate that is simply the average of the predator and prey occupants' traits. Again, as for the truncated normal distribution, other implementations can be explored. However, this is simply and straightforward to understand. A prey has an advantage if its `trait` value is low, a predator has an advantage if its `trait` value is high.

The final `combined_rate` is then compared with a randomly generated number $r$ between zero and one and the reaction takes place if $r$ is smaller than `combined_rate`.

In [ ]:
class TraitPredationBirthReaction(PredationBirthReaction):
    def decide(self, participants: list[TraitOccupant], rng: np.random.Generator):
        predator, prey = participants
        combined_rate = 0.5 * (predator.trait + prey.trait)
        return (combined_rate >= 1.0) or (rng.random() < combined_rate)

Next, as before, we set up the world. The following are the parameters of the simulation:
* `size`: As in the other examples, is the size of the 2D lattice and is specified as a tuple of two integers (the width and the height). It is good to keep this small to keep simulation times low, start with `(100, 100)`.
* `sigma`: This is the prey (species "B") birth rate. It regulates how quickly prey will reproduce.
* `mu`: This is the predator (species "A") death rate. It regulates how quickly predators are removed from the system.
* `nuA`: This is the predator (species "A") mutation rate (i.e. the width of the truncated normal distribution). It controls how quickly the species adapts.
* `nuB`: This is the prey (species "B") mutation rate (i.e. the width of the truncated normal distribution). It controls how quickly the species adapts.
* `hA` and `hB`: This is the hop rate, which in typically is left at a value of `1` since this will only rescale time. It could be varied for each species separately, such that prey e.g. more static than predators or vice-versa.
* `rhoA` and `rhoB`: This is the initial density of the predators (species "A") and prey (species "B"). It specifies how many occupants of each species will be present on each lattice site on average.
* `T`: Specifies the end time of the simulation, i.e. how many Monte Carlo steps will be performed.

In [ ]:
# The simulation parameters
size = (100, 100)
sigma: float = 0.2
mu: float = 0.2
nuA: float = 0.05
nuB: float = 0.05
hA: float = 1.0
hB: float = 1.0
rhoA: float = 0.1
rhoB: float = 0.1
T: int = 400

# Predator and prey species
A = EvolvingSpecies("A", nuA, 0.5)
B = EvolvingSpecies("B", nuB, 0.5)

# The reactions
reactions = {
    A: [
        TraitPredationBirthReaction(A, B, None),
        DeathReaction(A, mu),
    ],
    B: [
        BirthReaction(B, sigma),
    ]
}

# Initialization of the world
world = World(
    size=size,
    initial_densities={B: rhoB, A: rhoA},
    hops={A: Hop(A, hA), B: Hop(B, hB)},
    reactions=reactions,
)

Now we're ready to tun the simulations. We run `T` Monte Carlo steps (default is 400). How long this takes will depend on the parameter settings above and the hardware this is being run on.

At each time step, we record the distribution of trait values for each of the two species in the lists `frequencyA` and `frequencyB`.

In [ ]:
from tqdm.notebook import tqdm
tqdm.monitor_interval = 0

# Initialize list that will hold a view of the lattice state at each step.
# The first entry shows the initial distribution of species site occupants.
numbers = [world.asarrays()]
frequencyA = [A.trait_frequencies()]
frequencyB = [B.trait_frequencies()]

# Iterate over `T` time steps
# `tqdm` is a tool to display a progress bar.
for _ in tqdm(range(T), smoothing=0, desc="Simulation progress"):
    # Run a single time step
    world.step()
    # Save the state of the lattice (distribution of occupants)
    numbers.append(world.asarrays())
    frequencyA.append(A.trait_frequencies())
    frequencyB.append(B.trait_frequencies())

After this has successfully run, we'd like to visualize the results. The code in the cell below displays an interactive graph where you can see the state of the lattice at each step (left side), together with a plot of the species densities over time (right side). 

In [ ]:
# Display matplotlib figures inline (not the interactive notebook extension)
%matplotlib inline
# Load modules
from matplotlib.figure import Figure
from ipywidgets import interact
import numpy as np
from IPython.display import display

# Initialize figure and axes
fig = Figure(figsize=(8, 4), dpi=150)
ax = fig.add_axes([0, 0, 0.5, 1])
# We don't want any decorations on the axes.
ax.axis('off')
plot = fig.add_axes([0.6, 0.6, 0.4, 0.4])

# Extract densities from species distribution arrays
N = size[0]*size[1]
A_densities, B_densities = zip(*[(entry["A"].sum()/N, entry["B"].sum()/N) for entry in numbers])

# Plot the densities
plot.plot(A_densities, color="red", label="A")
plot.plot(B_densities, color="blue", label="B")
# Label the plot
plot.set_yscale("log", base=10)
plot.set_ylabel("Species density [occupants/site]")
plot.set_xlabel("Time [MC steps]")

freq_plot = fig.add_axes([0.6, 0.1, 0.4, 0.4])
freq_plot.set_ylabel("Trait frequency")
freq_plot.set_xlabel("Trait value")

# This code displays the correct view of the lattice given the MC step
mappable = None
vline = None
def show_image(MC_step=0):
    global mappable, vline, freqA, freqB  # Hackish, but easy, avoid global if possible.

    # Load array of lattice occupant numbers
    arrays = numbers[MC_step]
    # Create a red and blue image
    # `image` is a WxHx3 image where the three channels at the end correspond to red-green-blue
    image = np.zeros((size[0], size[1], 3), dtype=np.uint8)
    # Make a pixel that has at least one predator red, at least one prey blue and purple if both are present.
    # Leave the green channel alone.
    image[:, :, 0] = 255*(arrays["A"] > 0)
    image[:, :, 2] = 255*(arrays["B"] > 0)
    if mappable is None:
        # We need to create the mappable first, afterwards we can just load data into it.
        mappable = ax.imshow(image)
        # Similar for the vline indicating the MC step we're at.
        vline = plot.axvline(MC_step, color="black")
        x = (np.arange(51) + 0.5)/51
        freqA = freq_plot.plot(x, frequencyA[MC_step], '-', color='red')[0]
        freqB = freq_plot.plot(x, frequencyB[MC_step], '-', color='blue')[0]
    else:
        # Load data into mappable and vline.
        mappable.set_data(image)
        vline.set_xdata([MC_step])
        freqA.set_ydata(frequencyA[MC_step])
        freqB.set_ydata(frequencyB[MC_step])
        freq_plot.relim()
        freq_plot.autoscale_view(True, True, True)

    # Finally, display the updated figure.
    display(fig)

# THis lets us have a slider to select the MC step to display.
interact(show_image, MC_step=(0, len(numbers)-1))
None

With the default parameters, what you should see if you scroll through the MC steps above are the damped Lotka-Volterra oscillations. The trait frequencies in the lower right panel show how the predators (red) adapt to higher trait values (they become more proficient at hunting prey) while the prey (blue) adapt to lower trait values (they become better at evasion). 